In [1]:
from bs4 import BeautifulSoup as soup
import urllib
import pandas as pd
import os

In [27]:
url = "https://www.tapology.com/rankings/current-top-ten-lightweight-mma-fighters-155-pounds"
url2 = "https://www.tapology.com/rankings/current-top-ten-lightweight-mma-fighters-155-pounds?page=2&ranking=10"

In [28]:
req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
page = urllib.request.urlopen(req)
page_html =  page.read()
page.close()
req = urllib.request.Request(url2, headers={'User-Agent': 'Mozilla/5.0'})
page = urllib.request.urlopen(req)
page_html2 =  page.read()
page.close()

In [29]:
page_soup = soup(page_html, "html.parser")
page_soup2 = soup(page_html2,"html.parser")

In [30]:
names_html = page_soup.findAll("h1", {"class":"name"}) + page_soup2.findAll("h1", {"class":"name"})

In [31]:
type(names_html)

list

In [32]:
def quote_remover(name):
    first = name.find('"',0,len(name))
    second = name.find('"',first+1,len(name))
    if first == second:
        return name
    else:
        return name[:first] + name[second+2:]
names = []
for name in names_html:
    string = name.findAll("a")[0].text
    ret = quote_remover(string)
    names.append(ret)

In [8]:
#now we have the names of all 100 lightweights

In [9]:
names

['Dustin Poirier',
 'Justin Gaethje',
 'Charles Oliveira',
 'Michael Chandler',
 'Tony Ferguson',
 'Dan Hooker',
 'Beneil Dariush',
 'Conor McGregor',
 'Islam Makhachev',
 'Paul Felder',
 'Rafael dos Anjos',
 'Diego Ferreira',
 'Kevin Lee',
 'Al Iaquinta',
 'Drew Dober',
 'Thiago Moisés',
 'Arman Tsarukyan',
 'Gregor Gillespie',
 'Francisco Trinaldo',
 'Alexander Hernandez',
 'Rafael Fiziev',
 'Khabib Nurmagomedov',
 'Leonardo Santos',
 'Bobby Green',
 'Drakkar Klose',
 'Nasrat Haqparast',
 'Vinc Pichel',
 'Scott Holtzman',
 'Renato Moicano',
 'Brad Riddell',
 'Jim Miller',
 'Damir Ismagulov',
 'David Teymur',
 'Marc Diakiese',
 'Clay Guida',
 'Joel Alvarez',
 'Davi Ramos',
 'Magomed Mustafaev',
 'Matt Frevola',
 'Lando Vannata',
 'Khama Worthy',
 'John Makdessi',
 'Mairbek Taisumov',
 'Ottman Azaitar',
 'Mark O. Madsen',
 'Natan Schulte',
 'Mike Davis',
 'Christos Giagos',
 'Devonte Smith',
 'Joe Solecki',
 'Olivier Aubin-Mercier',
 'Damir Hadžović',
 'Michael Johnson',
 'Alan Patrick

In [10]:
striking_accuracies = []
grappling_accuracies = []
ages = []
heights = []
lreachs = []
reachs = []
countries = []
insta = []

w_percents = []

In [11]:
import time
count = 0
for name in names:
    name = name.replace(" ", "-")
    url = "https://www.ufc.com/athlete/" + name

    req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    try:
        page = urllib.request.urlopen(req)
    except:
        continue
    page_html =  page.read()
    page.close()

    page_soup = soup(page_html, "html.parser")

    twitter_link = page_soup.findAll("div", {"class":"c-hero__social-links"})[0]
    twitter_link = twitter_link.findAll("a", {"class": "c-hero__social-link"})
    twitter_flag=False
    for link in twitter_link:
        if "https://instagram.com" in link['href']:
            twitter_flag = True
            insta.append(link['href'])
    if twitter_flag==False:
        insta.append("Na")

    try:
        striking_accuracy =  int(page_soup.findAll("svg",{"class":"e-chart-circle"})[0].text.split("\n")[-2].split("%")[0])
        striking_accuracies.append(striking_accuracy)
    except:
        striking_accuracies.append(-1)

    try:
        grappling_accuracy = int(page_soup.findAll("svg",{"class":"e-chart-circle"})[1].text.split("\n")[-2].split("%")[0])
        grappling_accuracies.append(grappling_accuracy)

    except:
        grappling_accuracies.append(-1)

    row1 = page_soup.findAll("div", {"class":"c-bio__row--3col"})[0].text.split("\n")#.remove("")
    row1 = [x for x in row1 if x]
    row2 = page_soup.findAll("div", {"class":"c-bio__row--3col"})[1].text.split("\n")#.remove("")
    row2 = [x for x in row2 if x]

    try:
        reach = row2[3]
        reachs.append(reach)
    except:
        reachs.append(-1)

    try:
        leg_reach = row2[5]
        lreachs.append(leg_reach)
    except:
        lreachs.append(-1)

    try:
        height = row1[3]
        heights.append(height)
    except:
        heights.append(-1)

    try:
        age = row1[1]
        ages.append(age)
    except:
        ages.append(-1)

    try:
        country = page_soup.findAll("div", {"class":"c-bio__row--1col"})[1].text.split(",")[1][1:]#.split(",")[1]
        countries.append(country)
    except:
        countries.append("Na")

    record = page_soup.findAll("div", {"class":"c-hero__headline-suffix tz-change-inner"})[0].text.split("-")
    wins = int(record[0].split("\n")[-1])
    losses = int(record[1])
    try:
        draws = int(record[2][0])
    except:
        draws = 0
    #dv
    win_percent = wins/(wins+losses+draws)
    w_percents.append(win_percent)

    if count %10==0:
        time.sleep(5)
    count +=1
    print("Count ", count)


Count  1
Count  2
Count  3
Count  4
Count  5
Count  6
Count  7
Count  8
Count  9
Count  10
Count  11
Count  12
Count  13
Count  14
Count  15
Count  16
Count  17
Count  18
Count  19
Count  20
Count  21
Count  22
Count  23
Count  24
Count  25
Count  26
Count  27
Count  28
Count  29
Count  30
Count  31
Count  32
Count  33
Count  34
Count  35
Count  36
Count  37
Count  38
Count  39
Count  40
Count  41
Count  42
Count  43
Count  44
Count  45
Count  46
Count  47
Count  48
Count  49
Count  50
Count  51
Count  52
Count  53
Count  54
Count  55
Count  56
Count  57
Count  58
Count  59
Count  60
Count  61
Count  62
Count  63
Count  64
Count  65


IndexError: list index out of range

In [12]:
#time to plug this into twitter hashtag
len(countries) 

65

In [33]:
names = names[:65]

In [34]:
#check their followers

In [35]:
len(names)

65

In [16]:

follower_list = []
for url in insta:
    if url!="Na":
        req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        page = urllib.request.urlopen(req)
        page_html =  page.read()
        page.close()
        
        page_soup = soup(page_html, "html.parser")
        try:
            first = page_soup.findAll("meta")[7].text.find('"userInteractionCount":') + len('"userInteractionCount":')
            second = page_soup.findAll("meta")[7].text[first:].find("}")
            followers = int(page_soup.findAll("meta")[7].text[first:first+second].strip('""'))
            follower_list.append(followers)
        except:
            follower_list.append(-1)
        #print(followers)

In [17]:
len(follower_list)

65

In [36]:
countries = [x.replace("\n","") for x in countries]
data = {
    
    "Win-Percent": w_percents,
    "Name": names,
    "Leg-Reach": lreachs,
    "Reach": reachs,
    "Instagram-Followers": follower_list,
    "Age": ages,
    "Height": heights,
    "Grappling-Accuracy":grappling_accuracies,
    "Striking-Accuracy":striking_accuracies,
    "Birth-Country": countries
    
}
df = pd.DataFrame(data)
#len(lreachs)
#print(df)

In [37]:
df.to_csv(r'UFC_DATA_REG.csv', index = False)